In [161]:
%reset -sf

In [162]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [163]:
from pathlib import Path

path = Path('/kaggle/input/ecommerce-sales-dataset')
list(path.glob('*'))

from pandas import read_csv, DataFrame, Series

customers = read_csv(path / 'customer_details.csv')
baskets = read_csv(path / 'basket_details.csv')

[PosixPath('/kaggle/input/ecommerce-sales-dataset/customer_details.csv'),
 PosixPath('/kaggle/input/ecommerce-sales-dataset/basket_details.csv')]

In [164]:
customers

,customer_id,sex,customer_age,tenure
0,9798859,Male,44.0,93
1,11413563,Male,36.0,65
2,818195,Male,35.0,129
3,12049009,Male,33.0,58
4,10083045,Male,42.0,88
...,...,...,...,...
19995,12557307,Male,41.0,52
19996,12595961,Male,29.0,52
19997,12520991,Male,35.0,52
19998,12612719,Male,39.0,52


In [165]:
from pandas import to_datetime

baskets['basket_date'] = to_datetime(baskets['basket_date'], errors='coerce')
baskets

,customer_id,product_id,basket_date,basket_count
0,42366585,41475073,2019-06-19,2
1,35956841,43279538,2019-06-19,2
2,26139578,31715598,2019-06-19,3
3,3262253,47880260,2019-06-19,2
4,20056678,44747002,2019-06-19,2
...,...,...,...,...
14995,8336862,50977318,2019-05-26,2
14996,9500785,43862061,2019-05-26,2
14997,22787344,6041664,2019-05-26,2
14998,8221263,3597369,2019-05-26,2


In [166]:
# First of all, how many customers are inthe baskets data

cids = Series(customers['customer_id'].unique())
bids = Series(baskets['customer_id'].unique())

cids.isin(bids).sum()
bids.isin(cids).sum()

# So, out of al 15,000 basket data, only 64 has customers data...

64

64

In [167]:
baskets['dayofweek'] = baskets['basket_date'].dt.dayofweek
baskets['isweekend'] = 0
mask = (baskets['dayofweek']==5) | (baskets['dayofweek']==6)
baskets['isweekend'] = 0
baskets.loc[mask, 'isweekend'] = 1

baskets

,customer_id,product_id,basket_date,basket_count,dayofweek,isweekend
0,42366585,41475073,2019-06-19,2,2,0
1,35956841,43279538,2019-06-19,2,2,0
2,26139578,31715598,2019-06-19,3,2,0
3,3262253,47880260,2019-06-19,2,2,0
4,20056678,44747002,2019-06-19,2,2,0
...,...,...,...,...,...,...
14995,8336862,50977318,2019-05-26,2,6,1
14996,9500785,43862061,2019-05-26,2,6,1
14997,22787344,6041664,2019-05-26,2,6,1
14998,8221263,3597369,2019-05-26,2,6,1


In [168]:
from pandas import crosstab

crosstab(baskets['dayofweek'], baskets['isweekend'])

isweekend,0,1
dayofweek,,
0,3175,0
1,2732,0
2,1469,0
3,1023,0
4,1667,0
5,0,2207
6,0,2727


In [169]:
customers

,customer_id,sex,customer_age,tenure
0,9798859,Male,44.0,93
1,11413563,Male,36.0,65
2,818195,Male,35.0,129
3,12049009,Male,33.0,58
4,10083045,Male,42.0,88
...,...,...,...,...
19995,12557307,Male,41.0,52
19996,12595961,Male,29.0,52
19997,12520991,Male,35.0,52
19998,12612719,Male,39.0,52


In [170]:
baskets

,customer_id,product_id,basket_date,basket_count,dayofweek,isweekend
0,42366585,41475073,2019-06-19,2,2,0
1,35956841,43279538,2019-06-19,2,2,0
2,26139578,31715598,2019-06-19,3,2,0
3,3262253,47880260,2019-06-19,2,2,0
4,20056678,44747002,2019-06-19,2,2,0
...,...,...,...,...,...,...
14995,8336862,50977318,2019-05-26,2,6,1
14996,9500785,43862061,2019-05-26,2,6,1
14997,22787344,6041664,2019-05-26,2,6,1
14998,8221263,3597369,2019-05-26,2,6,1


In [171]:
# Identifying people that only goes shopping at weekends

in_week_cids = Series(baskets['customer_id'][baskets['isweekend']==0].unique())
in_weekend_cids = Series(baskets['customer_id'][baskets['isweekend']==1].unique())

In [172]:
mask = in_weekend_cids.isin(in_week_cids)
cids_only_at_weekends = in_weekend_cids[~mask]
cids_only_at_weekends

0        7751651
1       23462918
3       44364979
4        9912477
5       25083717
          ...   
4717     2468899
4718    12066558
4719     8336862
4720     9500785
4721    22787344
Length: 4474, dtype: int64

In [173]:
baskets['cids_only_at_weekends'] = 0
df_mask = baskets['customer_id'].isin(cids_only_at_weekends)
baskets.loc[df_mask, 'cids_only_at_weekends'] = 1

baskets

,customer_id,product_id,basket_date,basket_count,dayofweek,isweekend,cids_only_at_weekends
0,42366585,41475073,2019-06-19,2,2,0,0
1,35956841,43279538,2019-06-19,2,2,0,0
2,26139578,31715598,2019-06-19,3,2,0,0
3,3262253,47880260,2019-06-19,2,2,0,0
4,20056678,44747002,2019-06-19,2,2,0,0
...,...,...,...,...,...,...,...
14995,8336862,50977318,2019-05-26,2,6,1,1
14996,9500785,43862061,2019-05-26,2,6,1,1
14997,22787344,6041664,2019-05-26,2,6,1,1
14998,8221263,3597369,2019-05-26,2,6,1,0


In [174]:
# check id 8221263

check_cid = [8221263]
in_week_cids.isin(check_cid).sum()
in_weekend_cids.isin(check_cid).sum()

# customer shops at both in week as in the weekends
# or shops onyl in week...

1

1

In [175]:
# check 4912577

check_cid = [4912577]
in_week_cids.isin(check_cid).sum()
in_weekend_cids.isin(check_cid).sum()

# customer shops only at weekends

0

1

In [176]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
prods = ohe.fit_transform(baskets[['product_id']]).toarray()
prods

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [177]:
X = DataFrame(prods)
X

,0,1,2,3,4,5,6,7,8,9,...,13151,13152,13153,13154,13155,13156,13157,13158,13159,13160
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
X['customer_id'] = baskets['customer_id']
X

,0,1,2,3,4,5,6,7,8,9,...,13152,13153,13154,13155,13156,13157,13158,13159,13160,customer_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42366585
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35956841
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26139578
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3262253
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20056678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8336862
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9500785
14997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22787344
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8221263


In [179]:
X['y'] = baskets['cids_only_at_weekends']
X

,0,1,2,3,4,5,6,7,8,9,...,13153,13154,13155,13156,13157,13158,13159,13160,customer_id,y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42366585,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35956841,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26139578,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3262253,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20056678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8336862,1
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9500785,1
14997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22787344,1
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8221263,0


In [180]:
from pandas import merge

X_ = merge(X, 
           customers[['customer_id', 'sex', 'customer_age', 'tenure']], 
           left_on=['customer_id'], 
           right_on=['customer_id'],
           #how='outer'
           )
X_ = X_.drop(['customer_id'], axis=1)
X_

,0,1,2,3,4,5,6,7,8,9,...,13155,13156,13157,13158,13159,13160,y,sex,customer_age,tenure
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,Male,40.0,114
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,Male,30.0,63
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,Female,51.0,62
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,Male,42.0,117
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,Male,45.0,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,Male,33.0,52
68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,Male,46.0,37
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,Male,29.0,41
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1,Male,29.0,76


In [181]:
y = X_[['y']]
X_ = X_.drop('y', axis=1)

In [182]:
X_.shape, y.shape

((72, 13164), (72, 1))

In [183]:
# At this point, if I want to include the products one-hot-encoded as predictors, 
# I encounter much less rows that columns, representing a sparse estimation
# which I don't want to go to...
# So, I only estimate with sex 	customer_age 	tenure

In [184]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

lb = LabelBinarizer()
X_['sex'] = lb.fit_transform(X_['sex'])
X_

,0,1,2,3,4,5,6,7,8,9,...,13154,13155,13156,13157,13158,13159,13160,sex,customer_age,tenure
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,40.0,114
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,30.0,63
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,51.0,62
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,42.0,117
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,45.0,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,33.0,52
68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,46.0,37
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,29.0,41
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,29.0,76


In [185]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X_[['sex', 'customer_age', 'tenure']], y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

est = LogisticRegressionCV(cv=6)
est.fit(X_train, y_train)
pred = est.predict(X_test)

print(classification_report(y_test, pred))

((57, 3), (15, 3), (57, 1), (15, 1))

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegressionCV(cv=6)

              precision    recall  f1-score   support

           0       0.71      1.00      0.83        10
           1       1.00      0.20      0.33         5

    accuracy                           0.73        15
   macro avg       0.86      0.60      0.58        15
weighted avg       0.81      0.73      0.67        15



In [186]:
DataFrame(est.coef_, columns=['sex', 'customer_age', 'tenure']).T

,0
sex,0.004734
customer_age,0.002014
tenure,0.018989


### Male =>   adds to the probability that customer goes to weekend shopping
### Age =>    the older, the most likely to go weekend shopping
### Tenure => the higher, the most likely to go weekend shopping

In [187]:
# One way to fight the sparse estimation is to get rid of products that 
# this sub sample just did not buy

temp = X_.T.iloc[:13161, :]
temp.to_csv('temp.csv')

In [188]:
mask = temp==1
mask

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13156,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13157,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13158,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13159,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [189]:
mask.sum().sum()

72

In [190]:
# Apparently, all products were at some point purchased...
# So, I can't remove some that for this subset were not purchased at all
# So I am back with the sparse X data, where
# you have a lot of columns, but much oless rows...